In [8]:
import pandas as pd
import numpy as np
import os

# --- Manual Mapping for Team Name Standardization ---
# This is crucial for merging different datasets that might use different team names/abbreviations.
# This mapping covers common variations found in NBA data, including city/team names
# and some common abbreviations. Expand this if unique names are still found after running.
team_name_standardization = {
    'Atlanta Hawks': 'Atlanta Hawks', 'ATL': 'Atlanta Hawks',
    'Boston Celtics': 'Boston Celtics', 'BOS': 'Boston Celtics',
    'Brooklyn Nets': 'Brooklyn Nets', 'BRK': 'Brooklyn Nets', 'NJN': 'Brooklyn Nets', # New Jersey Nets
    'Charlotte Hornets': 'Charlotte Hornets', 'CHO': 'Charlotte Hornets', 'CHH': 'Charlotte Hornets', # Charlotte Bobcats, then Hornets
    'Charlotte Bobcats': 'Charlotte Hornets', # Map Bobcats to Hornets for consistency across eras if desired
    'Chicago Bulls': 'Chicago Bulls', 'CHI': 'Chicago Bulls',
    'Cleveland Cavaliers': 'Cleveland Cavaliers', 'CLE': 'Cleveland Cavaliers',
    'Dallas Mavericks': 'Dallas Mavericks', 'DAL': 'Dallas Mavericks',
    'Denver Nuggets': 'Denver Nuggets', 'DEN': 'Denver Nuggets',
    'Detroit Pistons': 'Detroit Pistons', 'DET': 'Detroit Pistons',
    'Golden State Warriors': 'Golden State Warriors', 'GSW': 'Golden State Warriors',
    'Houston Rockets': 'Houston Rockets', 'HOU': 'Houston Rockets',
    'Indiana Pacers': 'Indiana Pacers', 'IND': 'Indiana Pacers',
    'Los Angeles Clippers': 'Los Angeles Clippers', 'LAC': 'Los Angeles Clippers',
    'Los Angeles Lakers': 'Los Angeles Lakers', 'LAL': 'Los Angeles Lakers',
    'Memphis Grizzlies': 'Memphis Grizzlies', 'MEM': 'Memphis Grizzlies', 'VAN': 'Memphis Grizzlies', # Vancouver Grizzlies
    'Miami Heat': 'Miami Heat', 'MIA': 'Miami Heat',
    'Milwaukee Bucks': 'Milwaukee Bucks', 'MIL': 'Milwaukee Bucks',
    'Minnesota Timberwolves': 'Minnesota Timberwolves', 'MIN': 'Minnesota Timberwolves',
    'New Orleans Pelicans': 'New Orleans Pelicans', 'NOP': 'New Orleans Pelicans', 'NOK': 'New Orleans Pelicans', # New Orleans Hornets
    'New Orleans Hornets': 'New Orleans Pelicans', # Map Hornets to Pelicans for consistency across eras
    'New York Knicks': 'New York Knicks', 'NYK': 'New York Knicks',
    'Oklahoma City Thunder': 'Oklahoma City Thunder', 'OKC': 'Oklahoma City Thunder', 'SEA': 'Oklahoma City Thunder', # Seattle SuperSonics
    'Orlando Magic': 'Orlando Magic', 'ORL': 'Orlando Magic',
    'Philadelphia 76ers': 'Philadelphia 76ers', 'PHI': 'Philadelphia 76ers',
    'Phoenix Suns': 'Phoenix Suns', 'PHO': 'Phoenix Suns',
    'Portland Trail Blazers': 'Portland Trail Blazers', 'POR': 'Portland Trail Blazers',
    'Sacramento Kings': 'Sacramento Kings', 'SAC': 'Sacramento Kings',
    'San Antonio Spurs': 'San Antonio Spurs', 'SAS': 'San Antonio Spurs',
    'Toronto Raptors': 'Toronto Raptors', 'TOR': 'Toronto Raptors',
    'Utah Jazz': 'Utah Jazz', 'UTA': 'Utah Jazz',
    'Washington Wizards': 'Washington Wizards', 'WAS': 'Washington Wizards',
    'Washington Bullets': 'Washington Wizards' # Map Bullets to Wizards
}


# --- Step 1: Load and Merge Advanced Stats Files ---
advanced_stats_files = [f for f in os.listdir('.') if f.startswith('AdvancedStats') and f.endswith('.csv')]
advanced_stats_files.sort()

all_advanced_stats_df = pd.DataFrame()

print("Loading and cleaning AdvancedStats files...")
for file in advanced_stats_files:
    try:
        season_str = file.replace('AdvancedStats', '').replace('.csv', '')
        df_temp = pd.read_csv(file)

        # Check for the extra header row (based on your output, 'Rk' is usually first)
        # If 'Rk' is not in columns and 'Rk' is in the first row, then header=1
        if 'Rk' not in df_temp.columns and df_temp.iloc[0].astype(str).str.contains('Rk').any():
            df_temp = pd.read_csv(file, header=1)
        
        # Clean column names (remove leading/trailing spaces)
        df_temp.columns = df_temp.columns.str.strip()

        # Rename 'Tm' to 'Team' if it exists
        if 'Tm' in df_temp.columns:
            df_temp = df_temp.rename(columns={'Tm': 'Team'})
        
        # Clean team names in AdvancedStats (remove asterisks and apply standardization)
        df_temp['Team'] = df_temp['Team'].astype(str).str.replace('*', '', regex=False).str.strip()
        df_temp['Team'] = df_temp['Team'].replace(team_name_standardization)
        
        df_temp['Season'] = season_str
        
        # Drop problematic 'Unnamed' columns and 'PL▼' if they exist
        cols_to_drop = [col for col in df_temp.columns if 'Unnamed:' in col or col == 'PL▼']
        df_temp = df_temp.drop(columns=cols_to_drop, errors='ignore') # errors='ignore' prevents error if column not found
        
        all_advanced_stats_df = pd.concat([all_advanced_stats_df, df_temp], ignore_index=True)
        # print(f"Loaded and cleaned {file}") # Commented to reduce verbose output
    except Exception as e:
        print(f"Error loading or cleaning {file}: {e}")

print("\nAdvancedStats DataFrame Head (after cleaning):")
print(all_advanced_stats_df.head())
print("\nAdvancedStats DataFrame Info (after cleaning):")
all_advanced_stats_df.info()
print("\nUnique Teams in AdvancedStats (after cleaning):", sorted(all_advanced_stats_df['Team'].unique()))


# --- Step 2: Load and Clean Team Payroll Data ---
print("\nLoading and cleaning TeamPayroll2000-2025.csv...")
df_payroll = pd.read_csv('TeamPayroll2000-2025.csv')

# Rename columns for consistency
df_payroll = df_payroll.rename(columns={
    'team': 'Team',
    'seasonStartYear': 'SeasonStartYear',
    'payroll': 'Payroll',
    'inflationAdjPayroll': 'InflationAdjPayroll'
})

# Drop 'Unnamed: 0' if it exists
if 'Unnamed: 0' in df_payroll.columns:
    df_payroll = df_payroll.drop(columns=['Unnamed: 0'])

# Format 'Season' to 'YYYY-YYYY'
df_payroll['Season'] = df_payroll['SeasonStartYear'].astype(str) + '-' + (df_payroll['SeasonStartYear'] + 1).astype(str)

# Clean and convert 'Payroll' and 'InflationAdjPayroll' to numeric
for col in ['Payroll', 'InflationAdjPayroll']:
    # Remove '$' and ','
    df_payroll[col] = df_payroll[col].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
    # Convert to numeric, coerce errors to NaN
    df_payroll[col] = pd.to_numeric(df_payroll[col], errors='coerce')

# Apply team name standardization
df_payroll['Team'] = df_payroll['Team'].astype(str).str.strip().replace(team_name_standardization)

print("\nTeamPayroll DataFrame Head (after cleaning):")
print(df_payroll.head())
print("\nTeamPayroll DataFrame Info (after cleaning):")
df_payroll.info()
print("\nUnique Teams in Payroll (after cleaning):", sorted(df_payroll['Team'].unique()))


# --- Step 3: Load and Clean Team Revenue Data ---
print("\nLoading and cleaning TeamRevenue2000-2025.csv...")
df_revenue = pd.read_csv('TeamRevenue2000-2025.csv')

# Rename columns for consistency
df_revenue = df_revenue.rename(columns={
    'team': 'Team',
    'seasonStartYear': 'SeasonStartYear',
    'revenue': 'Revenue'
})

# Format 'Season' to 'YYYY-YYYY'
df_revenue['Season'] = df_revenue['SeasonStartYear'].astype(str) + '-' + (df_revenue['SeasonStartYear'] + 1).astype(str)

# Clean and convert 'Revenue' to numeric (e.g., "$157M" -> 157,000,000)
def clean_revenue(revenue_str):
    revenue_str = str(revenue_str).replace('$', '').strip()
    if 'M' in revenue_str:
        return float(revenue_str.replace('M', '')) * 1_000_000
    if 'B' in revenue_str: # In case there are Billion values
        return float(revenue_str.replace('B', '')) * 1_000_000_000
    return pd.to_numeric(revenue_str, errors='coerce') # Handle cases without M/B or other issues

df_revenue['Revenue'] = df_revenue['Revenue'].apply(clean_revenue)

# Apply team name standardization
df_revenue['Team'] = df_revenue['Team'].astype(str).str.strip().replace(team_name_standardization)

print("\nTeamRevenue DataFrame Head (after cleaning):")
print(df_revenue.head())
print("\nTeamRevenue DataFrame Info (after cleaning):")
df_revenue.info()
print("\nUnique Teams in Revenue (after cleaning):", sorted(df_revenue['Team'].unique()))


# --- Step 4: Merge All DataFrames ---
# Use an outer merge to see all possible combinations and identify non-merging rows if any
print("\nMerging dataframes...")
merged_df = pd.merge(all_advanced_stats_df, df_payroll[['Team', 'Season', 'Payroll', 'InflationAdjPayroll']],
                     on=['Team', 'Season'], how='left')
merged_df = pd.merge(merged_df, df_revenue[['Team', 'Season', 'Revenue']],
                     on=['Team', 'Season'], how='left')

print("\nMerged DataFrame Head (after full merge and initial cleaning):")
print(merged_df.head())
print("\nMerged DataFrame Info (after full merge and initial cleaning):")
merged_df.info()
print("\nMissing values after merge (Crucial Check!):")
print(merged_df.isnull().sum())

# --- Step 5: Define & Tag Superteams (Re-integrating Phase 1 after cleaning) ---
# Your Superteam list remains the same.
superteam_eras = {
    'Boston Celtics': (2007, 2012),
    'Miami Heat': (2010, 2014),
    'Cleveland Cavaliers': (2014, 2017),
    'Golden State Warriors': (2016, 2019),
    'Los Angeles Lakers': [(2003, 2004), (2012, 2013), (2021, 2023)],
    'Brooklyn Nets': [(2013, 2014), (2021, 2022)],
    'Oklahoma City Thunder': (2017, 2018),
    'Los Angeles Clippers': (2023, 2024),
    'Phoenix Suns': (2023, 2025)
}

merged_df['Team_Type'] = 'Balanced'

def is_superteam(row):
    team = row['Team']
    season_full_str = row['Season']
    
    try:
        season_start_year = int(season_full_str.split('-')[0])
    except (ValueError, IndexError):
        return 'Balanced' # Cannot parse season, default to Balanced

    if team in superteam_eras:
        eras = superteam_eras[team]
        if isinstance(eras, tuple):
            start_year, end_year = eras
            if start_year <= season_start_year <= end_year:
                return 'Superteam'
        elif isinstance(eras, list):
            for start_year, end_year in eras:
                if start_year <= season_start_year <= end_year:
                    return 'Superteam'
    return 'Balanced'

print("\nTagging Superteams...")
merged_df['Team_Type'] = merged_df.apply(is_superteam, axis=1)

print("\nSample of Merged DataFrame with Team_Type (after cleaning and tagging):")
# Assuming 'W' is Wins, we also need to decide on 'Playoff Wins'
# Based on the original prompt, 'Playoff Results' is a separate data type.
# For now, let's use 'W' for regular season wins, and acknowledge we still need to add 'Playoff_Wins'.
print(merged_df[['Season', 'Team', 'Team_Type', 'Payroll', 'W']].head(10))
print("\nDistribution of Team_Type (after cleaning and tagging):")
print(merged_df['Team_Type'].value_counts())

# --- Phase 3: Calculate Key Metrics ---
print("\n--- Phase 3: Calculating Key Metrics ---")

# Ensure necessary columns are numeric and handle potential NaNs (e.g., from merge failures)
# Fill NaN values for Payroll/Revenue with 0 or the median/mean if appropriate,
# but for cost/ROI calculation, missing values imply unknown cost/revenue, so NaN is fine.
# We will drop rows with NaN in critical columns if they are not meant to represent 0.
merged_df.dropna(subset=['W', 'Payroll', 'InflationAdjPayroll'], inplace=True)

# 1. Cost_Per_Win (using inflation-adjusted payroll for better time-series comparison)
# Handle division by zero if a team has 0 wins (results in infinity)
merged_df['Cost_Per_Win'] = merged_df['InflationAdjPayroll'] / merged_df['W']
# Fix: Remove inplace=True for replace as per FutureWarning
merged_df['Cost_Per_Win'] = merged_df['Cost_Per_Win'].replace([np.inf, -np.inf], np.nan)


# --- Placeholder for Playoff Wins ---
# !!! IMPORTANT: This 'Playoff_Wins' column is a placeholder.
# You need to fill this with actual playoff wins data for your analysis to be accurate.
# If 'PL' (Playoff Losses) from AdvancedStats is indicative of playoff appearance,
# you might use that to set a minimum 'Playoff_Wins' for teams that made playoffs,
# but actual wins require more specific data.
merged_df['Playoff_Wins'] = 0 # Initializing to 0
# You would load your actual playoff data and merge it here if available:
# df_playoffs = pd.read_csv('your_playoff_data.csv')
# merged_df = pd.merge(merged_df, df_playoffs[['Team', 'Season', 'Playoff_Wins']], on=['Team', 'Season'], how='left')


# 2. Cost_Per_Playoff_Win
# Handle division by zero (results in infinity)
merged_df['Cost_Per_Playoff_Win'] = merged_df['InflationAdjPayroll'] / merged_df['Playoff_Wins']
# Fix: Remove inplace=True for replace as per FutureWarning
merged_df['Cost_Per_Playoff_Win'] = merged_df['Cost_Per_Playoff_Win'].replace([np.inf, -np.inf], np.nan)


# 3. Win_Efficiency (Wins per dollar spent, scaled for readability)
# Divide by 1,000,000 to get wins per million dollars
merged_df['Win_Efficiency'] = merged_df['W'] / (merged_df['InflationAdjPayroll'] / 1_000_000)
# Fix: Remove inplace=True for replace as per FutureWarning
merged_df['Win_Efficiency'] = merged_df['Win_Efficiency'].replace([np.inf, -np.inf], np.nan)


# 4. Playoff_Efficiency (Playoff Wins per dollar spent, scaled)
merged_df['Playoff_Efficiency'] = merged_df['Playoff_Wins'] / (merged_df['InflationAdjPayroll'] / 1_000_000)
# Fix: Remove inplace=True for replace as per FutureWarning
merged_df['Playoff_Efficiency'] = merged_df['Playoff_Efficiency'].replace([np.inf, -np.inf], np.nan)


# 5. Revenue_Per_Win (Optional, requires 'Revenue' column)
# Handle division by zero
merged_df['Revenue_Per_Win'] = merged_df['Revenue'] / merged_df['W']
# Fix: Remove inplace=True for replace as per FutureWarning
merged_df['Revenue_Per_Win'] = merged_df['Revenue_Per_Win'].replace([np.inf, -np.inf], np.nan)


# --- Calculate League Averages for ROI comparison (for Win_ROI_vs_League_Average later) ---
# This is a prerequisite for 'Win_ROI_vs_League_Average'
# Calculate league average InflationAdjPayroll per season
league_avg_payroll_per_season = merged_df.groupby('Season')['InflationAdjPayroll'].mean().reset_index()
league_avg_payroll_per_season.rename(columns={'InflationAdjPayroll': 'League_Avg_Payroll'}, inplace=True)
merged_df = pd.merge(merged_df, league_avg_payroll_per_season, on='Season', how='left')

# Calculate league average Wins per season
league_avg_wins_per_season = merged_df.groupby('Season')['W'].mean().reset_index()
league_avg_wins_per_season.rename(columns={'W': 'League_Avg_Wins'}, inplace=True)
merged_df = pd.merge(merged_df, league_avg_wins_per_season, on='Season', how='left')

# Calculate League Average Win Efficiency (Wins per million dollars)
merged_df['League_Avg_Win_Efficiency'] = merged_df['League_Avg_Wins'] / (merged_df['League_Avg_Payroll'] / 1_000_000)

# 6. Win_ROI_vs_League_Average (Team Win Efficiency - League Avg Win Efficiency)
# This shows how much better/worse a team is in win efficiency compared to the league average for that season
merged_df['Win_ROI_vs_League_Average'] = merged_df['Win_Efficiency'] - merged_df['League_Avg_Win_Efficiency']


print("\nMerged DataFrame Head (after calculating new metrics):")
# Displaying key new metrics along with core info
print(merged_df[['Season', 'Team', 'Team_Type', 'W', 'Playoff_Wins',
                 'Payroll', 'InflationAdjPayroll', 'Revenue',
                 'Cost_Per_Win', 'Cost_Per_Playoff_Win', 'Win_Efficiency',
                 'Playoff_Efficiency', 'Revenue_Per_Win', 'Win_ROI_vs_League_Average']].head())

print("\nSummary statistics for new metrics by Team Type:")
print(merged_df.groupby('Team_Type')[['Cost_Per_Win', 'Cost_Per_Playoff_Win',
                                    'Win_Efficiency', 'Playoff_Efficiency',
                                    'Revenue_Per_Win', 'Win_ROI_vs_League_Average']].agg(['mean', 'median', 'std']))

print("\nMissing values after calculating metrics:")
print(merged_df[['Cost_Per_Win', 'Cost_Per_Playoff_Win', 'Win_Efficiency',
                 'Playoff_Efficiency', 'Revenue_Per_Win', 'Win_ROI_vs_League_Average']].isnull().sum())

Loading and cleaning AdvancedStats files...

AdvancedStats DataFrame Head (after cleaning):
    Rk                    Team   Age     W     L  PW    PL   MOV   SOS   SRS  \
0  1.0       San Antonio Spurs  29.5  58.0  24.0  63  19.0  7.76  0.16  7.92   
1  2.0        Sacramento Kings  27.1  55.0  27.0  57  25.0  5.82  0.25  6.07   
2  3.0               Utah Jazz  32.0  53.0  29.0  55  27.0  4.70  0.31  5.00   
3  4.0      Philadelphia 76ers  28.2  56.0  26.0  54  28.0  4.28 -0.65  3.64   
4  5.0  Portland Trail Blazers  30.1  50.0  32.0  53  29.0  4.20  0.33  4.52   

   ...  ORB%  FT/FGA  eFG%.1  TOV%.1  DRB%  FT/FGA.1               Arena  \
0  ...  27.1   0.267   0.444    12.9  73.7     0.182           Alamodome   
1  ...  27.2   0.230   0.460    14.5  70.7     0.181     ARCO Arena (II)   
2  ...  30.2   0.273   0.473    15.6  73.2     0.301        Delta Center   
3  ...  31.2   0.262   0.460    14.5  72.5     0.186  First Union Center   
4  ...  30.0   0.225   0.470    14.3  72.4     